In [1]:
import pandas as pd
import numpy as np
import os
import utils
from ast import literal_eval

In [37]:
out_filename = 'out.csv'
ppt_filename = 'PPT2.csv'

In [38]:
out_df = pd.read_csv(out_filename)
ppt = pd.read_csv(ppt_filename)
out_df = out_df.iloc[:2431]
out_df.head()

,Unnamed: 0,x,y,conf,cls_conf,cls_pred,file
0,0,151.701859,223.045929,0.976195,0.999687,1.0,data/custom/sliceTs/ts-LNDb-0006-0196.jpg
1,1,146.343155,225.865540,0.823491,0.997536,1.0,data/custom/sliceTs/ts-LNDb-0006-0196.jpg
2,2,168.036804,222.385742,0.887685,0.997042,1.0,data/custom/sliceTs/ts-LNDb-0006-0199.jpg
3,3,414.475555,253.854187,0.995986,0.998764,1.0,data/custom/sliceTs/ts-LNDb-0006-0211.jpg
4,4,411.358093,256.484802,0.972242,0.998506,1.0,data/custom/sliceTs/ts-LNDb-0006-0211.jpg


In [39]:
ppt.head()

,Unnamed: 0,0,orig shp,origin,t_wrld
0,0,ts-LNDb-0160-0208.jpg,"(349, 512, 660)","(-208.03807160195095, -296.20541828947677, -12...",ts-LNDb-0160-0208
1,1,ts-LNDb-0160-0211.jpg,"(349, 512, 660)","(-208.03807160195095, -296.20541828947677, -12...",ts-LNDb-0160-0211
2,2,ts-LNDb-0160-0214.jpg,"(349, 512, 660)","(-208.03807160195095, -296.20541828947677, -12...",ts-LNDb-0160-0214
3,3,ts-LNDb-0160-0217.jpg,"(349, 512, 660)","(-208.03807160195095, -296.20541828947677, -12...",ts-LNDb-0160-0217
4,4,ts-LNDb-0160-0220.jpg,"(349, 512, 660)","(-208.03807160195095, -296.20541828947677, -12...",ts-LNDb-0160-0220


In [40]:
files = out_df['file']

In [41]:
def fileToId(file_str):
    return int((file_str).split('-')[-2].lstrip('0'))

In [42]:
def fileToZ(file_str):
    return int((file_str).split('-')[-1].lstrip('0').strip('.')[0])

In [43]:
def stripPath(df):
    df['file'] = df['file'].apply(lambda x: x.split('/')[-1].split('.')[0])

In [44]:
stripPath(out_df)

In [45]:
out_df.head()

,Unnamed: 0,x,y,conf,cls_conf,cls_pred,file
0,0,151.701859,223.045929,0.976195,0.999687,1.0,ts-LNDb-0006-0196
1,1,146.343155,225.865540,0.823491,0.997536,1.0,ts-LNDb-0006-0196
2,2,168.036804,222.385742,0.887685,0.997042,1.0,ts-LNDb-0006-0199
3,3,414.475555,253.854187,0.995986,0.998764,1.0,ts-LNDb-0006-0211
4,4,411.358093,256.484802,0.972242,0.998506,1.0,ts-LNDb-0006-0211


In [60]:
ppt_trans = ppt['t_wrld']
out_files = out_df['file']

'(328, 512, 512)'

In [67]:
coords = []
for i in range(len(ppt)):
    o_shp = ppt['orig shp'][ppt_trans.apply(lambda x: x[:-5]) == out_files.apply(lambda x: x[:-5])[i]].values[0]
    xy = ((out_df[['x', 'y']].iloc[i].values / 512) * literal_eval(o_shp)[1:])
    z = fileToZ(files[i])
    xyz = np.array([xy[0], xy[1], z], dtype=int)
    trans_fn = ppt['t_wrld'][ppt_trans.apply(lambda x: x[:-5]) == out_files.apply(lambda x: x[:-5])[i]].values[0]
    file_origin = ppt['origin'][ppt_trans.apply(lambda x: x[:-5]) == out_files.apply(lambda x: x[:-5])[i]].values[0]
    xyz = utils.convertToWorldCoord(xyz,literal_eval(file_origin),np.load('./TransMTs2/' + trans_fn + '.npy'))
    coords.append(xyz)


In [73]:
ids = []
for file in out_df['file']:
    ids.append(fileToId(file))

In [69]:
confs = []
for i in range(len(ppt)):
    conf = out_df['conf'][i] * out_df['cls_conf'][i] if out_df['cls_pred'][i] == 1 else out_df['conf'][i] * (1 - out_df['cls_conf'][i])
    confs.append(conf)

In [70]:
sub_df = pd.DataFrame(coords, columns=['x','y','z'])
sub_df['LNDbID'] = pd.Series(ids)
sub_df['Nodule'] = pd.Series(confs)

In [71]:
sub_df.head()

,x,y,z,LNDbID,Nodule
0,-81.640625,-154.390625,-148.5,6,0.975890
1,-85.546875,-152.828125,-148.5,6,0.821462
2,-68.359375,-155.171875,-148.5,6,0.885060
3,123.828125,-130.953125,-147.5,6,0.994755
4,121.484375,-128.609375,-147.5,6,0.970789


In [72]:
sub_df.to_csv('predictedNodulesA.csv', columns=['LNDbID', 'x', 'y', 'z', 'Nodule'], index=None)